In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## Create JSON annotations from YOLO Labels

In [1]:
# prepare the data
# give the correct data path
!python ../src/faster_rcnn/data/prepare_data_faster_rcnn.py \
    --datasets ../testing_faster \
    --out-dir ../src/artifacts \
    --class-name polyp


[info] processing dataset: /Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging/testing_faster
[info] detected layout A under ../testing_faster
   train images: True → ../testing_faster/images/train
   train labels: True → ../testing_faster/labels/train
   val images:   True → ../testing_faster/images/val
   val labels:   True → ../testing_faster/labels/val
[info] scanning images in ../testing_faster/images/train
[info] found 2 images in ../testing_faster/images/train
[diag] ../testing_faster/images/train: images=2 matched_labels=2 boxes=2 skipped=0
[info] scanning images in ../testing_faster/images/val
[info] found 1 images in ../testing_faster/images/val
[diag] ../testing_faster/images/val: images=1 matched_labels=1 boxes=1 skipped=0
[info] wrote ../src/artifacts/train.json
[info] wrote ../src/artifacts/val.json
[info] wrote ../src/artifacts/roots_map.json


## Run the model WITHOUT evaluation

In [11]:
%cd /Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging
%env PYTHONPATH=src:$PYTHONPATH

!python -m faster_rcnn.training.train_faster_rcnn \
    --train-json src/artifacts/train.json \
    --val-json src/artifacts/val.json \
    --roots-map src/artifacts/roots_map.json \
    --images-root src/faster_rcnn/data/center_stratified/C3 \
    --epochs 1 \
    --batch-size 8 \
    --num-workers 0 \
    --img-size 640 \
    --train-augs medium \
    --num-classes 2 \
    --freeze-backbone 2 \
    --opt sgd \
    --lr-backbone 1e-4 \
    --lr-heads 5e-3 \
    --weight-decay 1e-4 \
    --lr-scheduler none \
    --device cpu
    

/Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging
env: PYTHONPATH=src:$PYTHONPATH
/Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging/src/faster_rcnn/augmentations/presets_faster_rcnn.py:155: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  A.Affine(
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging/src/faster_rcnn/training/train_faster_rcnn.py:263: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))
Train E1:   0%|                                                               | 0/1 [00:00<?, ?it/s]/Users/rebekaheichberg/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/torch/utils/data/dataloader.

## Run the Evaluation

In [4]:
#!/usr/bin/env python3
from pathlib import Path
import sys
import torch

# Add your src folder to Python path
root = Path('/Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging')
SRC = Path("/Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging/src")
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from faster_rcnn.data.data_loaders_faster_rcnn import build_train_val_loaders
from faster_rcnn.data.coco_eval_faster_rcnn import coco_map
from faster_rcnn.models.faster_rcnn_model import build_fasterrcnn

# --- paths ---
ckpt_path  = root / "runs" / "frcnn_polyp" / "epoch_001.pth"
train_json = root / "src" / "artifacts" / "train.json"
val_json   = root / "src" / "artifacts" / "val.json"
roots_map  = root / "src" / "artifacts" / "roots_map.json"
images_root = root / "testing_faster"

# --- setup model ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = build_fasterrcnn(num_classes=2, pretrained=False)
state = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state["model"])
model.to(device).eval()

# --- build validation loader ---
_, val_loader = build_train_val_loaders(
    train_json=str(train_json),
    val_json=str(val_json),
    roots_map=str(roots_map),
    images_root=str(images_root),
    img_size=832,        # affects TRAIN only; val uses build_val_augs
    train_augs="light",
    batch_size=2,
    num_workers=0,
)

# --- run COCO eval ---
metrics = coco_map(
    model,
    val_loader,
    device,
    coco_gt_json=str(val_json),
    verbose=True,     # prints COCO table at the end
    log_every=1,      # print per batch; set 5/10 if too chatty
)

print("\nFinal metrics:")
print(
    f"  mAP@[.5:.95] = {metrics['mAP_50_95']:.4f}\n"
    f"  mAP@0.5      = {metrics['mAP_50']:.4f}\n"
    f"  Recall       = {metrics['recall']:.4f}\n"
    f"  Precision    = {metrics['precision']:.4f}"
)


/Users/rebekaheichberg/Desktop/git-re/erdos-2025-colorectal-imaging/src/faster_rcnn/augmentations/presets_faster_rcnn.py:132: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  A.Affine(  # replaces ShiftScaleRotate


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


COCO eval:   0%|                                                              | 0/1 [00:00<?, ?it/s]/Users/rebekaheichberg/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
COCO eval: 100%|█████████████████████████████████████████████████████| 1/1 [02:19<00:00, 139.01s/it]

[batch 1/1] cumulative detections: 100
   top scores this batch: [0.78230035 0.70178086 0.6982    ]
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.029
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 |